In [ ]:
import pandas as pd
import numpy as np
import joblib

from preprocessing_transformers import *

In [ ]:
pipeline = joblib.load('preprocessing_feature_pipeline.joblib')

In [ ]:
def preprocess_data(df):
    if 'SalePrice' in df.columns:
        X = df.drop(columns='SalePrice')
        y = np.log(df['SalePrice'])
    else:
        X = df
        y = None

    X_transformed = pipeline.transform(X)

    return X_transformed, y

# 1. Training the Final Model

In [ ]:
train_df = pd.read_csv('train.csv')
X_train, y_train = preprocess_data(train_df)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [11] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
final_model = joblib.load('final_model.joblib')
final_model.fit(X_train, y_train)

StackingRegressor(estimators=[('svr',
                               SVR(C=1, degree=2, epsilon=0.01, gamma='auto')),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='rmse',
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importan...
                                            max_delta_step=None, max_depth=4,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=42, ...))],
                  final_estimator=MLPRegressor(activation='tanh', alpha=0.01,
                                               early_stopping=True,
                                               hidden_layer_sizes=(100, 50),
                                               max_iter=10000, random_state=42,
                                               solver='lbfgs'))

# 2. Predicting

In [ ]:
test_df = pd.read_csv('test.csv')
X_test, _ = preprocess_data(test_df)
y_pred = final_model.predict(X_test)
y_pred

array([11.70802597, 12.00934552, 12.14111342, ..., 12.0022723 ,
       11.64455031, 12.30897215])

In [ ]:
submission = {
    'Id': test_df['Id'],
    'SalePrice': np.exp(y_pred)
}
submission_df = pd.DataFrame(submission)
submission_df

,Id,SalePrice
0,1461,121543.313501
1,1462,164282.949373
2,1463,187421.135421
3,1464,195535.121603
4,1465,188882.991414
...,...,...
1454,2915,85686.776226
1455,2916,77175.725978
1456,2917,163125.040287
1457,2918,114068.030534


# 3. Exporting the Results

In [ ]:
submission_df.to_csv('submission.csv', index=False)